In [3]:
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser
import helpers
import re
import unidecode

In [4]:
items_dict = helpers.load_items()
interactions_train = helpers.load_interactions_df()

In [5]:
def normalize(item_title):
    return unidecode.unidecode(item_title.lower().strip().replace('.', '').replace(',', '').replace(';', '').replace('_', '').replace('?', '').replace('!', '').replace('"', '').replace('\'', '')).strip()

In [6]:
search_queries = list(set([normalize(x).strip() for x in interactions_train[interactions_train['event_type'] == 'search']['item_id'].dropna().unique()]))

In [7]:
schema = Schema(itemid=ID(stored=True), content=TEXT)
ix = create_in("data/index", schema)

In [8]:
%%time
writer = ix.writer()

keys = list(items_dict.keys())
for i in range(len(keys)):
    x = keys[i]
    writer.add_document(itemid=str(x), content=f"{normalize(items_dict[x]['title'])}")
    if i % 100000 == 0 and i > 0:
        print(f"Comiting... i = {i}")
        writer.commit()
        writer = ix.writer()
        print(f"Finished commit!")
writer.commit()

Comiting... i = 100000
Finished commit!
Comiting... i = 200000
Finished commit!
Comiting... i = 300000
Finished commit!
Comiting... i = 400000
Finished commit!
Comiting... i = 500000
Finished commit!
Comiting... i = 600000
Finished commit!
Comiting... i = 700000
Finished commit!
Comiting... i = 800000
Finished commit!
Comiting... i = 900000
Finished commit!
Comiting... i = 1000000
Finished commit!
Comiting... i = 1100000
Finished commit!
Comiting... i = 1200000
Finished commit!
Comiting... i = 1300000
Finished commit!
Comiting... i = 1400000
Finished commit!
Comiting... i = 1500000
Finished commit!
Comiting... i = 1600000
Finished commit!
Comiting... i = 1700000
Finished commit!
Comiting... i = 1800000
Finished commit!
Comiting... i = 1900000
Finished commit!
Comiting... i = 2000000
Finished commit!
Comiting... i = 2100000
Finished commit!
CPU times: user 14min 39s, sys: 3min 8s, total: 17min 48s
Wall time: 18min 35s


In [28]:
qs = [QueryParser("content", ix.schema).parse(search_queries[i]) for i in range(1, 2000)]

In [30]:
%%time
results = []
with ix.searcher() as searcher:
    for i in range(1, 200):#len(search_queries)):
        query = qs[i]#QueryParser("content", ix.schema).parse(search_queries[i])
        results.append(searcher.search(query, limit=1))
    #for r in results:
    #    print(r)

CPU times: user 10.2 s, sys: 984 ms, total: 11.1 s
Wall time: 11.4 s


In [26]:
results

[<Top 0 Results for And([Term('content', 'tinta'), Term('content', 'negra'), Term('content', 'universal'), Term('content', 'premium')]) runtime=0.07119369999963965>,
 <Top 1 Results for And([Term('content', 'relogio'), Term('content', 'bluetooth'), Term('content', 'inteligente'), Term('content', 'j10'), Term('content', 'calorias'), Term('content', 'pressao'), Term('content', 'whatsapp')]) runtime=0.15817689999948925>,
 <Top 1 Results for And([Term('content', 'gold'), Term('content', 'liss')]) runtime=0.023254300000189687>,
 <Top 1 Results for And([Term('content', 'skin'), Term('content', 'fortinite'), Term('content', 'ps4')]) runtime=0.019065600000431004>,
 <Top 0 Results for And([Term('content', 'adubo'), Term('content', 'npk'), Term('content', '151520')]) runtime=0.007988199999999779>,
 <Top 1 Results for And([Term('content', 'cadeirinha'), Term('content', 'bb')]) runtime=0.013670299999830604>,
 <Top 0 Results for And([Term('content', 'xenon'), Term('content', 'motor'), Term('content